In [1]:
import torch
# 向量模型下载
from modelscope import snapshot_download
import streamlit as st
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader, CSVLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.chains import LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.llms.base import LLM
from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.text_splitter import RecursiveCharacterTextSplitter
import re
from typing import Any, List, Optional
from langchain_community.vectorstores import Chroma

In [2]:
snapshot_download('AI-ModelScope/tao-8k', cache_dir='./')

'./AI-ModelScope/tao-8k'

In [6]:
import os   
def get_embedding(embedding_model_path):
    model_kwargs = {'device': 'cuda'}
    encode_kwargs = {'normalize_embeddings': True}  # set True to compute cosine similarity
    embeddings = HuggingFaceEmbeddings(
        model_name=embedding_model_path,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs,
    )
    return embeddings

def get_vectordb(file_path: str=None, persist_path: str=None, embedding_path=None):
    """
    返回向量数据库对象
    输入参数：
    question：
    llm:
    vectordb:向量数据库(必要参数),一个对象
    template：提示模版（可选参数）可以自己设计一个提示模版，也有默认使用的
    embedding：可以使用zhipuai等embedding，不输入该参数则默认使用 openai embedding，注意此时api_key不要输错
    """

    embedding = get_embedding(embedding_path)
    if os.path.exists(persist_path):  #持久化目录存在
        contents = os.listdir(persist_path)
        if len(contents) == 0:  #但是下面为空
            #print("目录为空")
            vectordb = create_db(file_path, persist_path, embedding)
        else:
            #print("目录不为空")
            vectordb = load_knowledge_db(persist_path, embedding)
    else: #目录不存在，从头开始创建向量数据库
        vectordb = create_db(file_path, persist_path, embedding)
        #presit_knowledge_db(vectordb)
        # vectordb = load_knowledge_db(persist_path, embedding)

    return vectordb

def create_db(embedding_model, file_path:str=None, persist_path:str=None):

    loaders_chinese = [
        PyPDFLoader("./knowledge/中国近现代史纲要：2023 年版(1).pdf"),
        PyPDFLoader("./knowledge/《习近平谈治国理政》第一卷.pdf"),
        PyPDFLoader("./knowledge/《习近平谈治国理政》第二卷.pdf"),
        PyPDFLoader("./knowledge/《习近平谈治国理政》第三卷.pdf"),
        PyPDFLoader("./knowledge/《习近平谈治国理政》第四卷.pdf")
        # 如果需要还可以加入其他文件
    ]
    docs = []
    for loader in loaders_chinese:
        pages = loader.load()
        print(len(pages))
        docs.extend(pages)
    CHUNK_SIZE = 200
    # 知识库中相邻文本重合长度
    OVERLAP_SIZE = 50
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=CHUNK_SIZE,
        chunk_overlap=OVERLAP_SIZE
    )
    split_docs = text_splitter.split_documents(docs)

    vectordb = Chroma.from_documents(
        documents=split_docs,
        embedding=embedding_model,
        persist_directory=persist_path  # 允许我们将persist_directory目录保存到磁盘上
    )
    vectordb.persist()
    return vectordb


def load_knowledge_db(persist_path, embedding):
    vectordb = Chroma(
        persist_directory=persist_path,
        embedding_function=embedding
    )
    return vectordb



tao_8k_embedding_model_path = "./AI-ModelScope/tao-8k"
tao_8k = get_embedding(tao_8k_embedding_model_path)
vectordb = create_db(tao_8k, persist_path="./vector_db/tao_8k")

No sentence-transformers model found with name ./AI-ModelScope/tao-8k. Creating a new one with mean pooling.


429
536
542
427
538


/tmp/ipykernel_374/1186819388.py:68: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [7]:
from typing import Any, List, Mapping, Optional, Dict
from langchain_core.callbacks.manager import CallbackManagerForLLMRun
from langchain_core.language_models.llms import LLM
from zhipuai import ZhipuAI

# 继承自 langchain_core.language_models.llms.LLM,用于调用 ZhipuAI 的语言模型服务
class ZhipuAILLM(LLM):
    # 默认选用 glm-4 模型
    model: str = "glm-4"
    # 温度系数
    temperature: float = 0.1
    # API_Key
    api_key: str = "652a160546149ef4e3ec0ff881beebfe.D3UaKuk7FmiUn9WQ"
    max_tokens: int = 2048

    # 定义 _call 方法：
    # 这个方法实现了实际的 API 调用逻辑：
    # 初始化 ZhipuAI 客户端。
    # 生成请求参数messages。
    # 调用 chat.completions.create 方法获取响应。
    # 返回响应中的内容，如果没有结果则返回错误信息。
    def _call(self, prompt: str, stop: Optional[List[str]] = None,
              run_manager: Optional[CallbackManagerForLLMRun] = None,
              **kwargs: Any):
        # 生成 GLM 模型请求参数的方法：
        # 生成 GLM 模型的请求参数 messages，包括系统消息和用户输入
        def gen_glm_params(prompt):
            '''
            构造 GLM 模型请求参数 messages
            请求参数：
                prompt: 对应的用户提示词
            '''
            messages = [{"role": "user", "content": prompt}]
            return messages

        client = ZhipuAI(
            api_key=self.api_key
        )

        messages = gen_glm_params(prompt)
        response = client.chat.completions.create(
            model=self.model,
            messages=messages,
            temperature=self.temperature,
            max_tokens=self.max_tokens
        )

        if len(response.choices) > 0:
            return response.choices[0].message.content
        return "generate answer error"

    # 定义属性方法：
    # _default_params：返回调用 API 的默认参数。
    # _llm_type：返回模型类型的字符串标识。
    # _identifying_params：返回模型的标识参数。
    # 首先定义一个返回默认参数的方法
    @property
    def _default_params(self) -> Dict[str, Any]:
        normal_params = {
            "temperature": self.temperature,
        }
        # print(type(self.model_kwargs))
        return {**normal_params}

    @property
    def _llm_type(self) -> str:
        return "Zhipu"

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        return {**{"model": self.model}, **self._default_params}


In [ ]:
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
# 导入PromptTemplate类，用于创建问题模板
from langchain.chains import RetrievalQA
# 导入RetrievalQA类，用于执行检索问答
import sys
# 导入sys模块，用于操作系统相关功能
from langchain_community.vectorstores import Chroma
from zhipuai import ZhipuAI


sys.path.append("../")
# 添加上级目录到模块搜索路径
# from qa_chain.model_to_llm import model_to_llm
# # 导入model_to_llm函数，用于将模型转换为大语言模型(LLM)
# from qa_chain.get_vectordb import get_vectordb
# # 导入get_vectordb函数，用于获取向量数据库

# 定义 QA_chain_self 类：
# 该类用于创建和管理一个问答链系统，不带历史记录，类的描述文档说明了各个参数的用途。
class QA_chain_self():
    # 类描述
    """
    不带历史记录的问答链
    - model：调用的模型名称
    - temperature：温度系数，控制生成的随机性
    - top_k：返回检索的前k个相似文档
    - file_path：建库文件所在路径
    - persist_path：向量数据库持久化路径
    - api_key：所有模型都需要
    - embeddings：使用的embedding模型
    - embedding_key：使用的embedding模型的秘钥（智谱或者OpenAI）
    - template：可以自定义提示模板，没有输入则使用默认的提示模板default_template_rq
    """

    # 默认的提示模板，用于构建问答的输入
    default_template_rq = """
    使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答案。最多使用三句话。尽量使答案简明扼要。总是在回答的最后说“谢谢你的提问！”。
    {context}
    问题: {question}
    有用的回答:"""

    template1 = """给你一个任务：给定一段文本，这段文本中第一段是题目，题目中会有括号，后面四行为A，B，C，D四个选项，最后一行是题目的答案。你需要做的是：将题目中的括号以及括号内的内容替换为答案并输出问题，注意不需要输出选项。
    不需要你检查指出题目的对错，不要输出除了以上任务之外的任何回答。
    这是你需要处理的文本：{question}
    """
    template2 = """使用以下背景知识来回答最后的问题。不要试图编造答案。尽量简明扼要地回答。
    背景知识：{context}
    问题：{query}"""


    # 构造函数，初始化类实例
    # 初始化向量数据库和LLM
    # 初始化 PromptTemplate 和 RetrievalQA 类
    def __init__(self, model: str, temperature: float = 0.0, top_k: int = 4, choiceproblem_file_path: str = None,
                 analysis_vectordb_file_path: str = None, persist_path: str = None, api_key: str = None,
                 embedding="zhipuai", embedding_path=None, template=default_template_rq):
        # 类属性初始化
        self.model = model
        self.temperature = temperature
        self.top_k = top_k
        # self.file_path = file_path
        self.persist_path = persist_path
        self.api_key = api_key
        self.embedding = embedding
        # self.embedding_key = embedding_key
        self.embedding_path = embedding_path
        self.template = template
        client = ZhipuAI(api_key="652a160546149ef4e3ec0ff881beebfe.D3UaKuk7FmiUn9WQ")
        # 加载已存在的向量数据库
        # self.choiceproblem_vectordb = get_vectordb(choiceproblem_file_path, persist_path + "/choiceproblem", embedding_path)
        self.analysis_vectordb = get_vectordb(analysis_vectordb_file_path, persist_path + "/tao_8k", embedding_path)
        self.llm = ZhipuAILLM()

        # # 初始化PromptTemplate和RetrievalQA类
        # self.QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"], template=self.template)
        # self.retriever = self.vectordb.as_retriever(search_type="similarity", search_kwargs={'k': self.top_k})
        # self.qa_chain = RetrievalQA.from_chain_type(llm=self.llm, retriever=self.retriever,
        #                                             return_source_documents=True,
        #                                             chain_type_kwargs={"prompt": self.QA_CHAIN_PROMPT})
        self.prompt1 = PromptTemplate(
            input_variables=["question"],
            template=self.template1
        )
        self.chain1 = LLMChain(
            llm=self.llm, prompt=self.prompt1
        )

        self.prompt2 = PromptTemplate(
            input_variables=["context", "query"],
            template=self.template2
        )
        self.chain2 = LLMChain(
            llm=self.llm, prompt=self.prompt2
        )


    # 提供问答功能的方法，根据用户问题调用问答链并返回结果
    def answer(self, query: str = None, temperature=None, top_k=4):
        """
        核心方法，调用问答链
        arguments:
        - question：用户提问
        """
        # 检查问题是否为空
        if len(query) == 0:
            return ""

        # 如果没有指定温度或top_k参数，则使用初始化时的值
        if temperature is None:
            temperature = self.temperature
        if top_k is None:
            top_k = self.top_k

        # sim_docs = self.choiceproblem_vectordb.similarity_search(query, k=1)
        # question = ""
        # for sim_doc in sim_docs:
        #     print(sim_doc.page_content)
        #     print("--------------")
        #     question = question + sim_doc.page_content+'\n'
        # response = self.chain1(question)
        # print(response)
        # print("---------\n")
        # context = response['text']
        context = ""
        related_analysis = self.analysis_vectordb.similarity_search(query, k=3)
        for related_doc in related_analysis:
            # print(type(related_doc))
            context = context + related_doc.page_content
        result = self.chain2({'context': context, 'query': query})
        # 调用问答链并返回结果
        # result = self.qa_chain({"query": question, "temperature": temperature, "top_k": top_k})
        return result

In [11]:

prompt: str  # 用户 prompt
model: str = "glm-4-flash"  # 使用的模型
temperature: float = 0.1  # 温度系数
if_history: bool = False  # 是否使用历史对话功能
# API_Key
api_key: str = "652a160546149ef4e3ec0ff881beebfe.D3UaKuk7FmiUn9WQ"
# Secret_Key
secret_key: str = None
# access_token
access_token: str = None
# APPID
appid: str = None
# APISecret
Spark_api_secret: str = None
# Secret_key
Wenxin_secret_key: str = None
# 数据库路径
db_path: str = "./vector_db"
# 源文件路径
file_path: str = "./中国近现代史纲要：2023 年版(1).pdf"
# prompt template
prompt_template: str = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答
案。最多使用三句话。尽量使答案简明扼要。总是在回答的最后说“谢谢你的提问！”。
{context}
问题: {question}
有用的回答:"""
# Template 变量
input_variables: list = ["context", "question"]
# Embdding
embedding: str = "m3e"
# Top K
top_k: int = 5
# embedding_key
embedding_key = "652a160546149ef4e3ec0ff881beebfe.D3UaKuk7FmiUn9WQ"

choiceproblem_file_path="./vector_db/choiceproblem"
analysis_vectordb_file_path="./vector_db/tao_8k"
embedding_path = "./AI-ModelScope/tao-8k"


def get_chain():
    return QA_chain_self(model=model, temperature=temperature, top_k=top_k,
                              choiceproblem_file_path=choiceproblem_file_path, analysis_vectordb_file_path=analysis_vectordb_file_path, persist_path=db_path,
                              api_key=api_key, embedding_path=embedding_path, template=prompt_template)


response = get_chain().answer(query="习近平总书记强调，我们党全面领导、长期执政，面临的最大挑战是什么？")
print(response)

No sentence-transformers model found with name ./AI-ModelScope/tao-8k. Creating a new one with mean pooling.
/tmp/ipykernel_374/1405062993.py:85: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  self.chain1 = LLMChain(
/tmp/ipykernel_374/1405062993.py:130: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = self.chain2({'context': context, 'query': query})


{'context': '响，我们党面临的执政环境仍然是复杂的，影响党的先进性、弱化党的纯洁性的因素也是\n复杂的。党的队伍和自身状况发生重大而深刻的变化，迫切要求提高党的建设质量、增强\n党组织的政治功能和组织功能。我们要坚持问题导向，保持战略定力，以“越是艰险越向\n前”的英雄气概和“狭路相逢勇者胜”的斗争精神，坚定不移抓下去。\n习近平指出，要坚持以党的政治建设为统领，坚决维护党中央权威和集中统一领导。404习近平强调，全面从严治党必须持之以恒、毫不动摇。受国际国内环境各种因素的影\n响，我们党面临的执政环境仍然是复杂的，影响党的先进性、弱化党的纯洁性的因素也是\n复杂的。党的队伍和自身状况发生重大而深刻的变化，迫切要求提高党的建设质量、增强\n党组织的政治功能和组织功能。我们要坚持问题导向，保持战略定力，以“越是艰险越向\n前”的英雄气概和“狭路相逢勇者胜”的斗争精神，坚定不移抓下去。习近平指出，我们党是生于忧患、成长于忧患、壮大于忧患的政党。正是一代代中国\n共产党人心存忧患、肩扛重担，才团结带领中国人民不断从胜利走向新的胜利。我国形势\n总的是好的，但我们前进道路上面临的困难和风险也不少。国内外环境发生了深刻变化，\n面对的矛盾和问题发生了深刻变化，发展阶段和发展任务发生了深刻变化，工作对象和工\n作条件发生了深刻变化，对我们党长期执政能力和领导水平的要求也发生了深刻变化。中', 'query': '习近平总书记强调，我们党全面领导、长期执政，面临的最大挑战是什么？', 'text': '习近平总书记强调，我们党全面领导、长期执政，面临的最大挑战是党的先进性和纯洁性受到的影响以及党的建设质量、党组织的政治功能和组织功能需要进一步增强的问题。在复杂多变的执政环境下，如何保持党的先进性和纯洁性，提升党的建设质量，增强党组织的政治功能和组织功能，是我们党面临的最大挑战。'}
